In [5]:
from pydantic import BaseModel
import requests
import json
import sqlite3
from glom import glom

from typing import Optional

from sqlmodel import Field, Session, SQLModel, create_engine
from sqlmodel.pool import StaticPool  

engine = create_engine(
        "sqlite://",  
        connect_args={"check_same_thread": False},
        poolclass=StaticPool,  
    )

class Team(SQLModel, table=True):
    id: Optional[str] = Field(default=None, primary_key=True)
    name: str
    nickname: str
    team_code: str
    division_id: str
    logo: str
    league: str = "PWHL"

class Standings(BaseModel):
    name: str
    team_code: str
    games_played: str
    points: str
    regulation_wins: str
    non_reg_wins: str
    losses: str
    non_reg_losses: str
    goals_for: str
    goals_against: str
    games_remaining: str
    percentage: str
    overall_rank: str
    rank: int

class Venue(BaseModel):
    id: str
    name: str
    url: str

class Game(BaseModel):
    id: str
    date_with_day: str
    home_goal_count: str
    visiting_goal_count: str
    home_result: str
    away_result: str
    status: str
    home_team_id: str
    away_team_id: str
    game_report_url: str
    game_sheet_url: str
    venue: Venue
    

URLS = {
    "teams": "https://lscluster.hockeytech.com/feed/index.php?feed=statviewfeed&view=teamsForSeason&season=2&key=694cfeed58c932ee&client_code=pwhl&site_id=2",
    "standings_regular": "https://lscluster.hockeytech.com/feed/index.php?feed=statviewfeed&view=teams&groupTeamsBy=league&context=overall&site_id=2&season=1&special=false&key=694cfeed58c932ee&client_code=pwhl&league_id=1&division=undefined&sort=points&lang=en",
    "standings_playoffs": "https://lscluster.hockeytech.com/feed/index.php?feed=statviewfeed&view=teams&groupTeamsBy=league&context=overall&site_id=2&season=2&special=false&key=694cfeed58c932ee&client_code=pwhl&league_id=1&division=undefined&sort=points&lang=en",
    "schedule": "https://lscluster.hockeytech.com/feed/index.php?feed=statviewfeed&view=schedule&team=-1&season=1&month=-1&location=homeaway&key=694cfeed58c932ee&client_code=pwhl&site_id=2&league_id=1&division_id=-1&lang=en",
}

SQLModel.metadata.create_all(engine)

/Users/dpage/Projects/notebooks/.venv/lib/python3.11/site-packages/sqlmodel/main.py:549: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Team, and will be replaced in the string-lookup table.
  DeclarativeMeta.__init__(cls, classname, bases, dict_, **kw)


InvalidRequestError: Table 'team' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [2]:
response = requests.get(URLS['teams'])
team_text = response.text[:-1]
team_text = team_text[1:]
external_data = json.loads(team_text)['teams']

teams = []
for d in external_data:
    if d['id'] != -1:
        team = Team(**d)
        teams.append(team)

with Session(engine) as session:  
    for t in teams:
        session.add(t)  

    session.commit()  

In [3]:
response = requests.get(URLS['standings_regular'])

standings_text = response.text[:-1]
standings_text = standings_text[1:]

standings_data = json.loads(standings_text)[0]['sections'][0]['data']
s_teams = []
for t in standings_data:
    s_teams.append(Standings(**t['row']))

s_teams[0]

Standings(name='PWHL Montreal', team_code='MTL', games_played='14', points='27', regulation_wins='6', non_reg_wins='3', losses='2', non_reg_losses='3', goals_for='35', goals_against='30', games_remaining='10', percentage='0.643', overall_rank='1', rank=1)

In [6]:
response = requests.get(URLS['schedule'])

schedule_text = response.text[:-1]
schedule_text = schedule_text[1:]

schedule_data = json.loads(schedule_text)[0]['sections'][0]['data']

games = []
for s in schedule_data:
    print(s)
    h_goals = s['row']['home_goal_count']
    a_goals = s['row']['visiting_goal_count']
    if int(h_goals) > int(a_goals):
        h_result = "win"
        a_result = "loss"
    elif int(h_goals) == int(a_goals):
        h_result = "tie"
        a_result = "tie"
    else:
        h_result = "loss"
        a_result = "win"
    games.append(Game(
        id=s['row']['game_id'],
        away_result=a_result, 
        home_result=h_result, 
        home_goal_count=h_goals, 
        visiting_goal_count=a_goals, 
        date_with_day=s['row']['date_with_day'],
        away_team_id=s['prop']['visiting_team_city']['teamLink'], #SQLModel association?
        home_team_id=s['prop']['home_team_city']['teamLink'],
        game_report_url=s['prop']['game_report']['link'],
        game_sheet_url=s['prop']['game_sheet']['link'],
        status=s['row']['game_status'],
        venue=Venue(id='-1',name=s['row']['venue_name'],url=glom(s, 'prop.venue_name.venueUrl', default=''))
    ))

{'prop': {'home_team_city': {'teamLink': '6'}, 'visiting_team_city': {'teamLink': '4'}, 'venue_name': {'venueUrl': 'https://www.mattamyathleticcentre.ca/'}, 'game_report': {'link': 'https://lscluster.hockeytech.com/game_reports/text-game-report.php?client_code=pwhl&game_id=2&lang_id=1'}, 'game_sheet': {'link': 'https://lscluster.hockeytech.com/game_reports/official-game-report.php?client_code=pwhl&game_id=2&lang_id=1'}, 'game_summary_long': {'link': '/game-summary/?game_id=2', 'gameLink': '2'}, 'game_center': {'link': '/game-center/?game_id=2', 'gameLink': '2'}}, 'row': {'game_id': '2', 'date_with_day': 'Mon, Jan 1', 'home_goal_count': '0', 'visiting_goal_count': '4', 'game_status': 'Final', 'home_team_city': 'Toronto', 'visiting_team_city': 'New York', 'venue_name': 'Mattamy Athletic Centre', 'game_report': 'Game Report', 'game_sheet': 'Game Sheet', 'game_summary_long': 'GAME SUMMARY', 'game_center': 'Game Center'}}
{'prop': {'home_team_city': {'teamLink': '5'}, 'visiting_team_city': 

ValueError: invalid literal for int() with base 10: '-'

In [ ]:
from sqlmodel import select

def select_teams():
    with Session(engine) as session:
        statement = select(Team).where(Team.team_code == "BOS")
        results = session.exec(statement)
        for team in results:
            print(team)

In [ ]:
select_teams()